## 1. Setup and Imports

In [ ]:
# Standard libraries
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Disable TensorFlow GPU BEFORE importing TF (RTX 5070 sm_120 not yet supported)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Add src to path
sys.path.insert(0, os.path.abspath('.'))

# Data manipulation
import numpy as np
import pandas as pd
from pathlib import Path

# Deep Learning - PyTorch (uses GPU)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Deep Learning - TensorFlow (for VGG16 baseline)
import tensorflow as tf
# Force TF to CPU using soft placement
with tf.device('/CPU:0'):
    pass  # TF will run VGG16 on CPU

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sklearn
from sklearn.metrics import classification_report, confusion_matrix

# Reload modules to pick up changes
import importlib
import src.classes.pancan_classifier
importlib.reload(src.classes.pancan_classifier)

# Custom modules
from src.classes.data_loader import FlipkartDataLoader, ImageDataset
from src.classes.pancan_classifier import PanCANClassifier, create_pancan_model
from src.classes.vgg16_baseline import VGG16Classifier
from src.classes.metrics_evaluator import MetricsEvaluator
from src.classes.model_comparison import ModelComparison
from src.classes.context_aggregation import FeatureImportanceAnalyzer

# Set device for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"TensorFlow: Will use CPU with soft device placement")

In [ ]:
# Configuration
CONFIG = {
    # Paths
    'dataset_path': Path('./dataset/Flipkart'),
    'models_path': Path('./models'),
    'reports_path': Path('./reports'),
    
    # Data
    'image_size': 224,
    'batch_size': 16,
    'test_size': 0.25,
    'val_size': 0.15,
    
    # Training
    'epochs': 20,
    'learning_rate': 1e-4,
    'weight_decay': 0.01,
    'patience': 5,
    
    # PanCAN specific
    'num_walk_orders': 3,
    'hidden_dim': 512,
    'dropout': 0.3,
    
    # Random seed
    'seed': 42
}

# Set seeds for reproducibility
np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(CONFIG['seed'])
tf.random.set_seed(CONFIG['seed'])

print("Configuration loaded successfully!")

## 2. Data Loading and Preprocessing

In [ ]:
# Initialize data loader
data_loader = FlipkartDataLoader(
    dataset_path=CONFIG['dataset_path'],
    random_seed=CONFIG['seed']
)

# Load data
df = data_loader.load_data()
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

In [ ]:
# Extract categories and encode labels
categories = data_loader.extract_categories(level=0)

# Validate images
df = data_loader.validate_images()

# Get class information
class_names = data_loader.class_names
num_classes = data_loader.num_classes

print(f"\nClasses ({num_classes}): {class_names}")

In [ ]:
# Display class distribution
class_dist = data_loader.get_class_distribution()
print("\nClass Distribution:")
print(class_dist)

# Visualize
fig = px.bar(
    x=class_dist.index,
    y=class_dist.values,
    title='Class Distribution in Flipkart Dataset',
    labels={'x': 'Category', 'y': 'Count'},
    color=class_dist.values,
    color_continuous_scale='Viridis'
)
fig.update_layout(showlegend=False, xaxis_tickangle=-45)
fig.show()

In [ ]:
# Split data
train_df, val_df, test_df = data_loader.split_data(
    test_size=CONFIG['test_size'],
    val_size=CONFIG['val_size']
)

print(f"\nTrain samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

In [ ]:
# Define transforms for PyTorch (PanCAN)
train_transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("Transforms defined successfully!")

In [ ]:
# Create PyTorch datasets
train_dataset = ImageDataset(train_df, transform=train_transform)
val_dataset = ImageDataset(val_df, transform=val_transform)
test_dataset = ImageDataset(test_df, transform=val_transform)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=0
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=0
)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

## 3. Model Initialization

### 3.1 PanCAN with ResNet101 Backbone

In [ ]:
# RTX 5070 (sm_120/Blackwell) - Use torch.compile() to JIT compile CUDA kernels
import os
os.environ['TORCH_LOGS'] = 'recompiles'  # Show compilation info

device = torch.device('cuda')
print(f"Device: {device}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

# Create models and compile them with Inductor/Triton backend
print("\nCreating PanCAN-ResNet101 with torch.compile()...")
pancan_resnet101 = create_pancan_model(
    backbone='resnet101',
    num_classes=num_classes,
    pretrained=True,
    num_walk_orders=3
).to(device)

# Compile with inductor backend (uses Triton for JIT kernel compilation)
pancan_resnet101 = torch.compile(pancan_resnet101, mode='reduce-overhead')

# Warm-up forward pass to trigger JIT compilation
print("Compiling kernels (first forward pass)...")
with torch.no_grad():
    dummy = torch.randn(2, 3, 224, 224, device=device)
    _ = pancan_resnet101(dummy)
print("✓ PanCAN-ResNet101 compiled successfully!")

total_params = sum(p.numel() for p in pancan_resnet101.parameters())
print(f"PanCAN-ResNet101: {total_params:,} params on CUDA")

# Create and compile PanCAN-ConvNeXt
print("\nCreating PanCAN-ConvNeXt with torch.compile()...")
pancan_convnext = create_pancan_model(
    backbone='convnext_tiny',
    num_classes=num_classes,
    pretrained=True,
    num_walk_orders=3
).to(device)

pancan_convnext = torch.compile(pancan_convnext, mode='reduce-overhead')

print("Compiling kernels (first forward pass)...")
with torch.no_grad():
    _ = pancan_convnext(dummy)
print("✓ PanCAN-ConvNeXt compiled successfully!")

total_params_cvt = sum(p.numel() for p in pancan_convnext.parameters())
print(f"PanCAN-ConvNeXt: {total_params_cvt:,} params on CUDA")

print(f"\n{'='*50}")
print(f"✓ PanCAN models: CUDA (torch.compile)")
print(f"✓ VGG16: CPU (TensorFlow)")
print(f"{'='*50}")

In [ ]:
# Create PanCAN model with ResNet101
pancan_resnet101 = create_pancan_model(
    num_classes=num_classes,
    backbone='resnet101',
    pretrained=True,
    num_walk_orders=CONFIG['num_walk_orders'],
    hidden_dim=CONFIG['hidden_dim'],
    dropout=CONFIG['dropout']
)
pancan_resnet101 = pancan_resnet101.to(device)

# Count parameters
total_params = sum(p.numel() for p in pancan_resnet101.parameters())
trainable_params = sum(p.numel() for p in pancan_resnet101.parameters() if p.requires_grad)

print(f"\nPanCAN-ResNet101:")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

### 3.2 PanCAN with ConvNeXt-Tiny Backbone

In [ ]:
# Create PanCAN model with ConvNeXt-Tiny
pancan_convnext = create_pancan_model(
    num_classes=num_classes,
    backbone='convnext_tiny',
    pretrained=True,
    num_walk_orders=CONFIG['num_walk_orders'],
    hidden_dim=CONFIG['hidden_dim'],
    dropout=CONFIG['dropout']
)
pancan_convnext = pancan_convnext.to(device)

# Count parameters
total_params_cvt = sum(p.numel() for p in pancan_convnext.parameters())
trainable_params_cvt = sum(p.numel() for p in pancan_convnext.parameters() if p.requires_grad)

print(f"\nPanCAN-ConvNeXt-Tiny:")
print(f"  Total parameters: {total_params_cvt:,}")
print(f"  Trainable parameters: {trainable_params_cvt:,}")

### 3.3 VGG16 Baseline (Mission 6)

In [ ]:
# Create VGG16 baseline model (force CPU to avoid RTX 5070 compatibility issues)
with tf.device('/CPU:0'):
    vgg16_model = VGG16Classifier(
        num_classes=num_classes,
        class_names=class_names,
        image_size=CONFIG['image_size'],
        trainable_layers=0  # Frozen backbone
    )

print(f"\nVGG16 Baseline (CPU):")
vgg16_model.model.summary()

## 4. Training Functions

In [ ]:
def train_pancan_epoch(model, train_loader, optimizer, criterion, device):
    """Train PanCAN model for one epoch."""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in train_loader:
        images = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(train_loader), correct / total


def validate_pancan(model, val_loader, criterion, device):
    """Validate PanCAN model."""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in val_loader:
            images = batch['pixel_values'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return total_loss / len(val_loader), correct / total, all_preds, all_labels


print("Training functions defined!")

In [ ]:
def train_pancan_model(model, train_loader, val_loader, config, device, model_name='pancan'):
    """Full training loop for PanCAN model."""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay']
    )
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=5, T_mult=2
    )
    
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    best_val_acc = 0
    patience_counter = 0
    
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")
    
    for epoch in range(config['epochs']):
        # Train
        train_loss, train_acc = train_pancan_epoch(
            model, train_loader, optimizer, criterion, device
        )
        
        # Validate
        val_loss, val_acc, _, _ = validate_pancan(
            model, val_loader, criterion, device
        )
        
        # Update scheduler
        scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Print progress
        print(f"Epoch {epoch+1}/{config['epochs']} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), config['models_path'] / f'{model_name}_best.pt')
        else:
            patience_counter += 1
            if patience_counter >= config['patience']:
                print(f"\nEarly stopping at epoch {epoch+1}")
                break
    
    print(f"\nBest validation accuracy: {best_val_acc:.4f}")
    return history

## 5. Train Models

### 5.1 Train PanCAN-ResNet101

In [ ]:
# Train PanCAN with ResNet101
history_resnet101 = train_pancan_model(
    pancan_resnet101,
    train_loader,
    val_loader,
    CONFIG,
    device,
    model_name='pancan_resnet101'
)

### 5.2 Train PanCAN-ConvNeXt

In [ ]:
# Train PanCAN with ConvNeXt-Tiny
history_convnext = train_pancan_model(
    pancan_convnext,
    train_loader,
    val_loader,
    CONFIG,
    device,
    model_name='pancan_convnext'
)

### 5.3 Train VGG16 Baseline

In [ ]:
# Prepare data generators for VGG16 (TensorFlow)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators from dataframes
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='main_category',
    target_size=(CONFIG['image_size'], CONFIG['image_size']),
    batch_size=CONFIG['batch_size'],
    class_mode='sparse'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='main_category',
    target_size=(CONFIG['image_size'], CONFIG['image_size']),
    batch_size=CONFIG['batch_size'],
    class_mode='sparse'
)

In [ ]:
# Compile VGG16
vgg16_model.compile(
    optimizer='adam',
    learning_rate=CONFIG['learning_rate']
)

# Train VGG16
history_vgg16 = vgg16_model.train(
    train_generator,
    val_generator,
    epochs=CONFIG['epochs'],
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=CONFIG['patience'],
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3
        )
    ]
)

## 6. Evaluation and Comparison

In [ ]:
# Initialize metrics evaluator
evaluator = MetricsEvaluator(class_names=class_names)

# Dictionary to store results
results = {}

In [ ]:
# Evaluate PanCAN-ResNet101
pancan_resnet101.load_state_dict(
    torch.load(CONFIG['models_path'] / 'pancan_resnet101_best.pt')
)
criterion = nn.CrossEntropyLoss()

_, test_acc_r101, preds_r101, labels_r101 = validate_pancan(
    pancan_resnet101, test_loader, criterion, device
)

results['PanCAN-ResNet101'] = evaluator.compute_all_metrics(
    np.array(labels_r101), np.array(preds_r101)
)

print("\n" + "="*60)
print("PanCAN-ResNet101 Results")
print("="*60)
print(f"Test Accuracy: {results['PanCAN-ResNet101']['accuracy']:.4f}")
print(f"Macro F1: {results['PanCAN-ResNet101']['macro_f1']:.4f}")
print(f"Weighted F1: {results['PanCAN-ResNet101']['weighted_f1']:.4f}")

In [ ]:
# Evaluate PanCAN-ConvNeXt
pancan_convnext.load_state_dict(
    torch.load(CONFIG['models_path'] / 'pancan_convnext_best.pt')
)

_, test_acc_cvt, preds_cvt, labels_cvt = validate_pancan(
    pancan_convnext, test_loader, criterion, device
)

results['PanCAN-ConvNeXt'] = evaluator.compute_all_metrics(
    np.array(labels_cvt), np.array(preds_cvt)
)

print("\n" + "="*60)
print("PanCAN-ConvNeXt Results")
print("="*60)
print(f"Test Accuracy: {results['PanCAN-ConvNeXt']['accuracy']:.4f}")
print(f"Macro F1: {results['PanCAN-ConvNeXt']['macro_f1']:.4f}")
print(f"Weighted F1: {results['PanCAN-ConvNeXt']['weighted_f1']:.4f}")

In [ ]:
# Evaluate VGG16
test_generator = val_datagen.flow_from_dataframe(
    test_df,
    x_col='image_path',
    y_col='main_category',
    target_size=(CONFIG['image_size'], CONFIG['image_size']),
    batch_size=CONFIG['batch_size'],
    class_mode='sparse',
    shuffle=False
)

# Get predictions
preds_vgg16_prob = vgg16_model.model.predict(test_generator)
preds_vgg16 = np.argmax(preds_vgg16_prob, axis=1)
labels_vgg16 = test_generator.classes

results['VGG16'] = evaluator.compute_all_metrics(
    labels_vgg16, preds_vgg16, preds_vgg16_prob
)

print("\n" + "="*60)
print("VGG16 Baseline Results")
print("="*60)
print(f"Test Accuracy: {results['VGG16']['accuracy']:.4f}")
print(f"Macro F1: {results['VGG16']['macro_f1']:.4f}")
print(f"Weighted F1: {results['VGG16']['weighted_f1']:.4f}")

## 7. Comparative Analysis

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Model': ['VGG16 (Mission 6)', 'PanCAN-ResNet101', 'PanCAN-ConvNeXt'],
    'Accuracy': [
        results['VGG16']['accuracy'],
        results['PanCAN-ResNet101']['accuracy'],
        results['PanCAN-ConvNeXt']['accuracy']
    ],
    'Macro F1': [
        results['VGG16']['macro_f1'],
        results['PanCAN-ResNet101']['macro_f1'],
        results['PanCAN-ConvNeXt']['macro_f1']
    ],
    'Weighted F1': [
        results['VGG16']['weighted_f1'],
        results['PanCAN-ResNet101']['weighted_f1'],
        results['PanCAN-ConvNeXt']['weighted_f1']
    ],
    'Cohen Kappa': [
        results['VGG16']['cohen_kappa'],
        results['PanCAN-ResNet101']['cohen_kappa'],
        results['PanCAN-ConvNeXt']['cohen_kappa']
    ]
})

print("\n" + "="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)
print(comparison_df.to_string(index=False))

In [ ]:
# Visualize comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Accuracy & F1 Comparison', 'Per-Class F1 Score']
)

# Bar chart for main metrics
metrics_to_plot = ['Accuracy', 'Macro F1', 'Weighted F1']
colors = ['#636EFA', '#EF553B', '#00CC96']

for i, model in enumerate(['VGG16 (Mission 6)', 'PanCAN-ResNet101', 'PanCAN-ConvNeXt']):
    fig.add_trace(
        go.Bar(
            name=model,
            x=metrics_to_plot,
            y=[comparison_df[comparison_df['Model']==model][m].values[0] for m in metrics_to_plot],
            marker_color=colors[i]
        ),
        row=1, col=1
    )

# Per-class F1
for i, (model_name, result) in enumerate([('VGG16', results['VGG16']),
                                           ('PanCAN-R101', results['PanCAN-ResNet101']),
                                           ('PanCAN-CvT', results['PanCAN-ConvNeXt'])]):
    fig.add_trace(
        go.Bar(
            name=model_name,
            x=class_names,
            y=result['per_class_f1'],
            marker_color=colors[i],
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_layout(
    title='Model Comparison: PanCAN vs VGG16',
    barmode='group',
    height=500
)
fig.show()

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models_data = [
    ('VGG16', labels_vgg16, preds_vgg16),
    ('PanCAN-ResNet101', labels_r101, preds_r101),
    ('PanCAN-ConvNeXt', labels_cvt, preds_cvt)
]

for ax, (name, y_true, y_pred) in zip(axes, models_data):
    cm = confusion_matrix(y_true, y_pred, normalize='true')
    sns.heatmap(
        cm, annot=True, fmt='.2f', cmap='Blues',
        xticklabels=class_names, yticklabels=class_names,
        ax=ax
    )
    ax.set_title(f'{name}\nNormalized Confusion Matrix')
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(CONFIG['reports_path'] / 'figures' / 'confusion_matrices.png', dpi=150)
plt.show()

## 8. Feature Importance Analysis (PanCAN)

In [ ]:
# Initialize feature importance analyzer
analyzer = FeatureImportanceAnalyzer(pancan_resnet101, device)

# Get scale importance weights
scale_weights = pancan_resnet101.cross_scale.scale_weights
scale_weights_normalized = torch.softmax(scale_weights, dim=0).detach().cpu().numpy()

print("\nScale Importance Weights:")
for i, w in enumerate(scale_weights_normalized):
    print(f"  Scale {i+1}: {w:.4f} ({w*100:.1f}%)")

In [ ]:
# Visualize scale importance
fig = px.bar(
    x=[f'Scale {i+1}' for i in range(4)],
    y=scale_weights_normalized,
    title='PanCAN Scale Importance Weights (Learned)',
    labels={'x': 'Feature Scale', 'y': 'Weight'},
    color=scale_weights_normalized,
    color_continuous_scale='Viridis'
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Visualize attention maps for sample images
sample_batch = next(iter(test_loader))
sample_images = sample_batch['pixel_values'][:4].to(device)
sample_labels = sample_batch['labels'][:4]

# Get attention maps
with torch.no_grad():
    output_dict = pancan_resnet101(sample_images, return_features=True)

print(f"\nSample predictions:")
preds = output_dict['logits'].argmax(dim=-1)
for i in range(4):
    print(f"  Image {i+1}: True={class_names[sample_labels[i]]}, Pred={class_names[preds[i]]}")

## 9. Training History Visualization

In [ ]:
# Plot training history
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'PanCAN-ResNet101 Loss', 'PanCAN-ResNet101 Accuracy',
        'PanCAN-ConvNeXt Loss', 'PanCAN-ConvNeXt Accuracy'
    ]
)

# ResNet101
epochs = range(1, len(history_resnet101['train_loss']) + 1)
fig.add_trace(go.Scatter(x=list(epochs), y=history_resnet101['train_loss'], name='Train Loss', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=history_resnet101['val_loss'], name='Val Loss', line=dict(color='red', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=history_resnet101['train_acc'], name='Train Acc', line=dict(color='blue')), row=1, col=2)
fig.add_trace(go.Scatter(x=list(epochs), y=history_resnet101['val_acc'], name='Val Acc', line=dict(color='red', dash='dash')), row=1, col=2)

# ConvNeXt
epochs_cvt = range(1, len(history_convnext['train_loss']) + 1)
fig.add_trace(go.Scatter(x=list(epochs_cvt), y=history_convnext['train_loss'], name='Train Loss', line=dict(color='green'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=list(epochs_cvt), y=history_convnext['val_loss'], name='Val Loss', line=dict(color='orange', dash='dash'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=list(epochs_cvt), y=history_convnext['train_acc'], name='Train Acc', line=dict(color='green'), showlegend=False), row=2, col=2)
fig.add_trace(go.Scatter(x=list(epochs_cvt), y=history_convnext['val_acc'], name='Val Acc', line=dict(color='orange', dash='dash'), showlegend=False), row=2, col=2)

fig.update_layout(title='Training History', height=600)
fig.show()

## 10. Save Results

In [ ]:
# Save comparison results
comparison_df.to_csv(CONFIG['reports_path'] / 'model_comparison_results.csv', index=False)

# Save detailed results
import json
with open(CONFIG['reports_path'] / 'detailed_results.json', 'w') as f:
    # Convert numpy arrays to lists for JSON serialization
    results_serializable = {}
    for model, metrics in results.items():
        results_serializable[model] = {
            k: v if not isinstance(v, np.ndarray) else v.tolist()
            for k, v in metrics.items()
        }
    json.dump(results_serializable, f, indent=2)

print("\nResults saved to:")
print(f"  - {CONFIG['reports_path'] / 'model_comparison_results.csv'}")
print(f"  - {CONFIG['reports_path'] / 'detailed_results.json'}")

## 11. Conclusion

### Key Findings

1. **Performance**: PanCAN with [best backbone] achieves **X%** improvement in Macro F1 over VGG16 baseline

2. **Architecture Benefits**:
   - Multi-order random walks capture global context
   - Cross-scale aggregation leverages hierarchical features
   - Fewer parameters while maintaining/improving performance

3. **Interpretability**:
   - Scale attention weights provide insights into feature importance
   - Compatible with Grad-CAM for spatial localization

### Recommendations

- PanCAN is suitable for production deployment with proper optimization
- ConvNeXt backbone offers best efficiency-performance trade-off
- Further improvements possible with larger datasets and ensemble methods

In [ ]:
print("\n" + "="*60)
print("POC COMPLETE - Mission 8")
print("="*60)
print(f"\nBest Model: PanCAN-{['ResNet101', 'ConvNeXt'][np.argmax([results['PanCAN-ResNet101']['macro_f1'], results['PanCAN-ConvNeXt']['macro_f1']])]}")
print(f"Best Macro F1: {max(results['PanCAN-ResNet101']['macro_f1'], results['PanCAN-ConvNeXt']['macro_f1']):.4f}")
print(f"Improvement over VGG16: {(max(results['PanCAN-ResNet101']['macro_f1'], results['PanCAN-ConvNeXt']['macro_f1']) - results['VGG16']['macro_f1'])*100:.2f}%")